<a href="https://colab.research.google.com/github/lephuocdat2000/AI-Tempo-Run/blob/main/Augment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/AITempoRun/SynthText

/content/gdrive/MyDrive/AITempoRun/SynthText


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 90 kB 8.2 MB/s 
     |████████████████████████████████| 13.0 MB 84 kB/s 
     |████████████████████████████████| 17.3 MB 125 kB/s 
     |████████████████████████████████| 26.6 MB 28 kB/s 
     |████████████████████████████████| 2.0 MB 42.5 MB/s 
     |████████████████████████████████| 11.4 MB 22.0 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 194 kB 58.8 MB/s 
     |████████████████████████████████| 511 kB 51.5 MB/s 
     |████████████████████████████████| 24.8 MB 63 kB/s 
     |████████████████████████████████| 481 kB 70.0 MB/s 
     |████████████████████████████████| 7.9 MB 14.1 MB/s 
  Created wheel for tornado: filename=tornado-6.0.2-cp37-cp37m-linux_x86_64.whl size=423898 sha256=d531068f2874fb65e4f7b7a92fbff2975bfe5ec201521ec60b5f59af244f050b
  Stored in directory: /root/.c

In [ ]:
!python gen.py

In [ ]:
# Author: Ankush Gupta
# Date: 2015

"""
Visualize the generated localization synthetic
data stored in h5 data-bases
"""
from __future__ import division
import os
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt 
import h5py 
from common import *
import cv2


def viz_textbb(text_im, charBB_list, wordBB, alpha=1.0):
    """
    text_im : image containing text
    charBB_list : list of 2x4xn_i bounding-box matrices
    wordBB : 2x4xm matrix of word coordinates
    """
    plt.close(1)
    plt.figure(1,figsize=(15,15))
    plt.imshow(text_im)
    H,W = text_im.shape[:2]
    # # plot the character-BB:
    # for i in range(len(charBB_list)):
    #      bbs = charBB_list[i]
    #      ni = bbs.shape[-1]
    #      for j in range(ni):
    #          bb = bbs[:,:,j]
    #          bb = np.c_[bb,bb[:,0]]
    #          plt.plot(bb[0,:], bb[1,:], 'r', alpha=alpha/2)
  
    # plot the word-BB:
    for i in range(wordBB.shape[-1]):
        bb = wordBB[:,:,i]
        bb = np.c_[bb,bb[:,0]]
        plt.plot(bb[0,:], bb[1,:], 'g', alpha=alpha)
        # visualize the indiv vertices:
        vcol = ['r','g','b','k']
        for j in range(4):
            plt.scatter(bb[0,j],bb[1,j],color=vcol[j])        

    plt.gca().set_xlim([0,W-1])
    plt.gca().set_ylim([H-1,0])
    plt.show(block=False)

def main(db_fname):
    db = h5py.File(db_fname, 'r')
    dsets = sorted(db['data'].keys())
    print ("total number of images : ", colorize(Color.RED, len(dsets), highlight=True))
    start = 4321
    for k in dsets:
        rgb = db['data'][k][...]
        rgb = cv2.cvtColor(rgb,cv2.COLOR_RGB2BGR)
        cv2.imwrite('/content/gdrive/MyDrive/AITempoRun/SynthText/dataset/images/train_'+str(start)+'.jpg',rgb)
        charBB = db['data'][k].attrs['charBB']
        wordBB = db['data'][k].attrs['wordBB']
        txt = db['data'][k].attrs['txt']
        txt = [txt[i].replace("\n"," ").strip().split(" ") for i in range(len(txt))]
        txt_lst = []
        for i in txt:
           for j in i:
              if j!='': txt_lst.append(j)
      
        #viz_textbb(rgb, [charBB], wordBB)

        #write file
        f=open('/content/gdrive/MyDrive/AITempoRun/SynthText/dataset/labels/train_'+str(start)+'.txt',"w")
        for i in range(wordBB.shape[-1]):
               bb = wordBB[:,:,i]
               coordinate = ""
               for j in range(0,4):
                  coordinate += str(int(bb[0][j])) +","+str(int(bb[1][j]))+","
               coordinate += txt_lst[i]
               f.write(coordinate+"\n")
        f.close()
        start+=1    
           
        print ("image name        : ", colorize(Color.RED, k, bold=True))
        #print ("  ** no. of chars : ", colorize(Color.YELLOW, charBB.shape[-1]))
        #print ("  ** no. of words : ", colorize(Color.YELLOW, wordBB.shape[-1]))
        #print ("  ** text         : ", colorize(Color.GREEN, txt_lst))

        # if 'q' in input("next? ('q' to exit) : "):
        #     break
    db.close()
    print(start)
if __name__=='__main__':
    main('results/SynthText.h5')

